# Step 1 - Gathering Raw Session Data into Local Data Lake
Retrieving raw JSON data for Oracle OpenWorld & CodeOne Session Catalog into local JSON Files to surmount unavailability, latency and format limitations. 

The session catalog for Oracle OpenWorld 2018 is available through a public web site, at: https://events.rainfocus.com/widget/oracle/oow18/catalogoow18? . The session catalog for the co-located CodeOne conference is published at https://events.rainfocus.com/widget/oracle/oow18/catalogcodeone18?. These websites use a common backend API to search for and retrieve details of conference sessions. The session catalog data is available from this REST API whose root endpoint is https://events.rainfocus.com/api/search.

A typical call to this API uses Headers (rfwidgetid: 'KKA8rC3VuZo5clh8gX5Aq07XFonUTLyU',rfapiprofileid: 'uGiII5rYGOjoHXOZx0ch4r7f1KzFC0zd') and Form values to retrieve specific session information. 

A bare bone API call in Python looks like this:
```
import requests

url = "https://events.rainfocus.com/api/search"

querystring = {"search.sessiontype":"1522435540042001BxTD"}

payload = "------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"size\"\r\n\r\n50\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"type\"\r\n\r\nsession\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"from\"\r\n\r\n30\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW--"
headers = {
    'content-type': "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    'rfapiprofileid': "uGiII5rYGOjoHXOZx0ch4r7f1KzFC0zd",
    'rfwidgetid': "KKA8rC3VuZo5clh8gX5Aq07XFonUTLyU",
    'cache-control': "no-cache",
    }

response = requests.request("POST", url, data=payload, headers=headers, params=querystring)

print(response.text)
```
This request returns all sessions at CodeOne of type Developer Session.

Execute the next cell to see this code in action and check the response from the API:

In [24]:
import requests

url = "https://events.rainfocus.com/api/search"

querystring = {"search.sessiontype":"1522435540042001BxTD"}

payload = "------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"size\"\r\n\r\n50\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"type\"\r\n\r\nsession\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"from\"\r\n\r\n30\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW--"
headers = {
    'content-type': "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    'rfapiprofileid': "uGiII5rYGOjoHXOZx0ch4r7f1KzFC0zd",
    'rfwidgetid': "KKA8rC3VuZo5clh8gX5Aq07XFonUTLyU",
    'cache-control': "no-cache",
    'Postman-Token': "dadd9f76-6a7f-41ed-8f31-04359976c622"
    }

response = requests.request("POST", url, data=payload, headers=headers, params=querystring)
#print first 4000 characters of response
print(response.text[:4000])

{"responseCode":"0","responseMessage":"Success","total":398,"numItems":50,"from":30,"size":50,"items":[{"sessionID":"1525864756644001hv0J","externalID":"1525864756644001hv0J","code":"DEV5118","abbreviation":"DEV5118","title":"Better Software, Faster: Principles of Continuous Delivery and DevOps","abstract":"The \u003ci\u003eWall Street Journal\u003c/i\u003e already mentioned it in 2011: “Software is eating the world.” Nowadays, every company is an IT company. Product owners and other business representatives seeing their competitors release new features to end users every day are demanding the same from their own software teams. How do you measure up to this heavy pressure as an IT organization? How do you quickly make changes to software systems in fast-paced environments without losing your grip on quality? How do you build and test software in such a way that it\u0027s always in a releasable state? This session explains the principles of continuous delivery and DevOps. You’ll leave 

Our challenge is to retrieve the complete data on all sessions for both conferences - CodeOne and Oracle OpenWorld. With the code in the previous cell as starting point - we can create a Python program that pulls session data from the (semi-)public API and stores it in a local file in our *data lake* (folder /datalake). Note that we have to make an API call for each session type and for both conferences in order to gather all data.

The next cell contains the first part of this program. Note that all imports and variables defined in this cell have - when executed - a lasting effect throughout the Jupyter Notebook session. In other words: variable `OOW_rfapiprofileid` is defined in the next cell and will be available to all subsequent cells in the notebook.

In [25]:
import requests # module for making HTTP requests, for example invoking REST APIs
import pandas as pd # module for working with Panel Data; A panel is a 3D container of data. The term Panel data is derived from econometrics and is partially responsible for the name pandas
import json # module for working with JSON data 
import math 
url = "https://events.rainfocus.com/api/search" # rooturl for session catalog
dataLake = "datalake/" # file system directory used for storing the gathered data

#headers required in each API call
querystring = {"search.sessiontype":"<session type specific code>"} 
payload = "------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"size\"\r\n\r\n50\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"type\"\r\n\r\nsession\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"from\"\r\n\r\n{0}\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW--"

headers = {
    'content-type': "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    'rfapiprofileid': "<conference specific profile id>",
    'rfwidgetid': "<conference specific widgetid>",
    'cache-control': "no-cache"
}

# OOW
OOW_rfapiprofileid ="K9HkkU5es180AVTifYUgYembIKJ15CMM"
OOW_rfwidgetid= "VEsNDADSTFH5azU4dH1QslO3lhpQTy4U"

# CodeOne
CodeOne_rfapiprofileid= "uGiII5rYGOjoHXOZx0ch4r7f1KzFC0zd"
CodeOne_rfwidgetid =  "KKA8rC3VuZo5clh8gX5Aq07XFonUTLyU"

Conference session data should be collected for all session types. The list of session types is defined in the next cell in a Map - associating the session type abbreviation to the identifier recognized by the API.

This list was compiled by searching one by one for every available session type in the Web UI and inspecting the HTTP request that was sent from the Web UI to the backend API.

In [26]:
sessionTypes =   {
      'BOF': '1518466139979001dQkv'
    , 'BQS': 'bqs'
    , 'BUS': '1519240082595001EpMm'
    , 'CAS': 'casestudy'
    , 'DEV': '1522435540042001BxTD'
    , 'ESS': 'ess'
    , 'FLP': 'flp'
    , 'GEN': 'general'
    , 'HOL': 'hol'
    , 'HOM': 'hom'
    , 'IGN': 'ignite'
    , 'KEY': 'option_1508950285425'
    , 'MTE':'1523906206279002QAu9'
    , 'PKN': '1527614217434001RBfj'
    , 'PRO': '1518464344082003KVWZ'
    , 'PRM': '1518464344082002KM3k'
    , 'TRN': '1518464344082001KHky'
    , 'SIG': 'sig'
    , 'THT': 'ts'
    , 'TLD': '1537894888625001RriS'
    , 'TIP': '1517517756579001F3CR'
    , 'TUT': 'tutorial'
    , 'TRN': '1518464344082001KHky'
}
                
print("Mapping of Session Type to header-code", sessionTypes)



Mapping of Session Type to header-code {'BOF': '1518466139979001dQkv', 'BQS': 'bqs', 'BUS': '1519240082595001EpMm', 'CAS': 'casestudy', 'DEV': '1522435540042001BxTD', 'ESS': 'ess', 'FLP': 'flp', 'GEN': 'general', 'HOL': 'hol', 'HOM': 'hom', 'IGN': 'ignite', 'KEY': 'option_1508950285425', 'MTE': '1523906206279002QAu9', 'PKN': '1527614217434001RBfj', 'PRO': '1518464344082003KVWZ', 'PRM': '1518464344082002KM3k', 'TRN': '1518464344082001KHky', 'SIG': 'sig', 'THT': 'ts', 'TLD': '1537894888625001RriS', 'TIP': '1517517756579001F3CR', 'TUT': 'tutorial'}


Two functions are defined to make API calls for a conference and specific session type. The session details are retrieved in batched of 50; for any session type for which there are more than 50 sessions of a certain type, multiple calls have to be made.

Function `loadSessionDataForSessionType` coordinates the API calling - handling the batches for example and converting the relevant section from the raw JSON response into a Pandas DataFrame object. This function leverages `loadSessionDataForSessionTypeStartingAt` to make the actual HTTP requests to the Session Catalog API - for session details for a specific conference and a session type from a certain offset.

In [27]:
def loadSessionDataForSessionTypeStartingAt(conference, sessionType, startingAt):
    querystring = {"search.sessiontype":sessionType} 
    if conference == "codeone":
       headers['rfapiprofileid'] = CodeOne_rfapiprofileid
       headers['rfwidgetid'] = CodeOne_rfwidgetid
    else :
       headers['rfapiprofileid'] = OOW_rfapiprofileid
       headers['rfwidgetid'] = OOW_rfwidgetid
    # make the HTTP request
    response = requests.request("POST", url, data=payload.format(startingAt), headers=headers, params=querystring)
    return response.text

def loadSessionDataForSessionType(conference, sessionType):
    tempDict = json.loads(loadSessionDataForSessionTypeStartingAt(conference,  sessionType,0))
    sl= tempDict ["sectionList"][0]
    sessions = sl["items"]
    total = sl.get("total",-1) # indicates the total number of sessions of type sessionType
    received = sl['numItems']
    startingAt = sl['from']
    print(" ** Received for session type ",sessionType," a set of ", received , "items, out of a total of ", total);
    if total > received:
        # figure out how many calls are required to get all data for the sessions of this type
        requestTotal = math.ceil(total/50)
        print ("number of additional requests = ",requestTotal-1)
        for k in range(1,requestTotal):
            # for subsequent request, the response does not contain the nested sectionList element
            tempDict = json.loads(loadSessionDataForSessionTypeStartingAt( conference, sessionType, k*50))
            items = tempDict['items']

            received = tempDict['numItems']
            startingAt = tempDict['from']
            print(" ** Received from " , startingAt," set of ", received , "items, out of a total of ", total);
            #combine sessions with new set of items
            sessions = sessions + items
    #the JSON object is converted to a Dict; this Dict contains scalar values such as strings and more complex values such as Arrarys and nested Dicts)
    ss = pd.DataFrame(sessions)
    return ss;

Let's make a single call for one session type at CodeOne, to get a feel for what's happening. The result is written to a JSON data file in our data lake (i.e. folder `./datalake`). 

In [29]:
#get session details for session type BOF (Birds of a Feather) at the CodeOne conference
key = 'BOF'
ss = loadSessionDataForSessionType('codeone',sessionTypes[key])
# write details to a JSON file called oow2018-sessions_codeone_BOF.json in the datalake (folder ./datalake)
ss.to_json("{0}oow2018-sessions_codeone_{1}.json".format(dataLake, key), force_ascii=False)
# show the first five entries in the data frame:
ss.head(5)

{"responseCode":"0","responseMessage":"Success","sections":true,"sectionList":[{"sectionId":"1","sectionTitle":"","total":43,"numItems":43,"from":0,"size":50,"items":[{"sessionID":"1526592270835001iepf","externalID":"1526592270835001iepf","code":"BOF6086","abbreviation":"BOF6086","title":"Accelerating Big Data Analytics on Oracle Cloud","abstract":"Data is the essential fuel to every analytic prob
 ** Received for session type  1518466139979001dQkv  a set of  43 items, out of a total of  43


,abbreviation,abstract,allowDoubleBooking,attributevalues,code,codeParts,code_id,es_metadata_id,event,eventCode,...,type_displayorder,type_displayorder_string,useDoubleBooking,useWaitingList,videos,viewAccess,viewAccessPublic,viewFileAccess,waitlistAccess,waitlistLimit
0,BOF6086,Data is the essential fuel to every analytic p...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",BOF6086,"{'alpha0': 'BOF', 'numeric1': '6086'}",bof6086,1526592270835001iepf,Oracle OpenWorld,oow18,...,9999.0,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
1,BOF4977,Microservices are independent—sure. Complex tr...,0,"[{'value': 'Intermediate', 'attributevalue_id'...",BOF4977,"{'alpha0': 'BOF', 'numeric1': '4977'}",bof4977,1525702590325001nlzi,Oracle OpenWorld,oow18,...,9999.0,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
2,BOF5402,This session takes the audience through the ri...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",BOF5402,"{'alpha0': 'BOF', 'numeric1': '5402'}",bof5402,15259703423020012GvZ,Oracle Code One,oow18,...,9999.0,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
3,BOF4909,Leveraging the developer session “Evolutionary...,0,"[{'value': 'Advanced', 'attributevalue_id': '1...",BOF4909,"{'alpha0': 'BOF', 'numeric1': '4909'}",bof4909,1525400274646001Otyb,Oracle Code One,oow18,...,9999.0,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
4,BOF5039,"<i>Cloud-native</i>. It’s a great term, one th...",0,"[{'value': 'Intermediate', 'attributevalue_id'...",BOF5039,"{'alpha0': 'BOF', 'numeric1': '5039'}",bof5039,15257821699770013WRE,Oracle Code One,oow18,...,9999.0,9999,True,0,NaN,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0


You can now check in folder `./datalake` for a file called oow2018-sessions_codeone_BOF.json that was created just now and contains in JSON format the fairly raw data on Bird of a Feathers sessions.

## Gather all raw session data into the Data Lake

The next cell will gather all session details for all types of sessions at both conferences (Oracle OpenWorld and CodeOne). This involves a substantial number of HTTP calls to the public API and a sizable data volume. 

*Note: Running the next cell can take several minutes.* 

In [30]:
#For both conferences, loop over all session types and invoke function loadSessionDataForSessionType
print("Loading CodeOne details")
for key, value in sessionTypes.items():
    ss = loadSessionDataForSessionType('codeone', value)
    ss.to_json("{0}oow2018-sessions_codeone_{1}.json".format(dataLake, key), force_ascii=False)

print("Loading OOW details")
for key, value in sessionTypes.items():
    ss = loadSessionDataForSessionType('oow', value)
    ss.to_json("{0}oow2018-sessions_oow_{1}.json".format(dataLake, key), force_ascii=False)
    


Loading CodeOne details
{"responseCode":"0","responseMessage":"Success","sections":true,"sectionList":[{"sectionId":"1","sectionTitle":"","total":43,"numItems":43,"from":0,"size":50,"items":[{"sessionID":"1526592270835001iepf","externalID":"1526592270835001iepf","code":"BOF6086","abbreviation":"BOF6086","title":"Accelerating Big Data Analytics on Oracle Cloud","abstract":"Data is the essential fuel to every analytic prob
 ** Received for session type  1518466139979001dQkv  a set of  43 items, out of a total of  43
{"responseCode":"0","responseMessage":"Success","sections":true,"sectionList":[{"sectionId":"1","sectionTitle":"","total":0,"numItems":0,"from":0,"size":50,"items":[]}],"attributes":[{"id":"weaflag","name":"WEA Flag","displayorder":1.0,"dataForm":"1503936308L8ef50df64","published":1.0,"viewAccessPublic":true,"viewAccess":[],"scheduleAccess":[],"values":[{"rf_attributevalue_id":"1500909747808001zGV
 ** Received for session type  bqs  a set of  0 items, out of a total of  0
{"r

{"responseCode":"0","responseMessage":"Success","sections":true,"sectionList":[{"sectionId":"1","sectionTitle":"","total":3,"numItems":3,"from":0,"size":50,"items":[{"sessionID":"1536717591822001FQGj","externalID":"1536717591822001FQGj","code":"KEY6651","abbreviation":"KEY6651","title":"Keynote: Groundbreakers: The Code Avengers","abstract":"You’ll Find Me Coding in the Cloud\u003cbr\u003e\n“The t
 ** Received for session type  option_1508950285425  a set of  3 items, out of a total of  3
{"responseCode":"0","responseMessage":"Success","sections":true,"sectionList":[{"sectionId":"1","sectionTitle":"","total":32,"numItems":32,"from":0,"size":50,"items":[{"sessionID":"1537515393902001QjIG","externalID":"1537515393902001QjIG","code":"MTE6736","abbreviation":"MTE6736","title":"Building Data Lakes with Kafka + Object Stores + ADW","abstract":"Come to this session for an informal present
 ** Received for session type  1523906206279002QAu9  a set of  32 items, out of a total of  32
{"response

{"responseCode":"0","responseMessage":"Success","sections":true,"sectionList":[{"sectionId":"1","sectionTitle":"","total":307,"numItems":50,"from":0,"size":50,"items":[{"sessionID":"1524162399613001CH4A","externalID":"1524162399613001CH4A","code":"CAS4312","abbreviation":"CAS4312","title":"185-Year-Old Startup","abstract":"As a newly created standalone department from the Great Lakes Water Authori
 ** Received for session type  casestudy  a set of  50 items, out of a total of  307
number of additional requests =  6
{"responseCode":"0","responseMessage":"Success","total":307,"numItems":50,"from":50,"size":50,"items":[{"sessionID":"1521570063472001jT5F","externalID":"1521570063472001jT5F","code":"CAS1750","abbreviation":"CAS1750","title":"Cox Automotive Moves Mission-Critical Workloads to Oracle Cloud Infrastructure ","abstract":"Cox Automotive supports more than 40,000 clients in more than 100 countries. Its 
 ** Received from  50  set of  50 items, out of a total of  307
{"responseCode

{"responseCode":"0","responseMessage":"Success","sections":true,"sectionList":[{"sectionId":"1","sectionTitle":"","total":375,"numItems":50,"from":0,"size":50,"items":[{"sessionID":"1521743878357001gLy5","externalID":"1521743878357001gLy5","code":"PRO2816","abbreviation":"PRO2816","title":"10 Things You Should Know About Oracle and Kubernetes","abstract":"This session covers 10 things everyone int
 ** Received for session type  1518464344082003KVWZ  a set of  50 items, out of a total of  375
number of additional requests =  7
{"responseCode":"0","responseMessage":"Success","total":375,"numItems":50,"from":50,"size":50,"items":[{"sessionID":"1521382335986001FKnw","externalID":"1521382335986001FKnw","code":"PRO1518","abbreviation":"PRO1518","title":"Case Study: 40 Million Tables in MySQL 8.0","abstract":"Two great features (among others) in MySQL 8.0 are the complete removal of *.frm files from the data dictionary and th
 ** Received from  50  set of  50 items, out of a total of  375
{"r

{"responseCode":"0","responseMessage":"Success","total":203,"numItems":3,"from":200,"size":50,"items":[{"sessionID":"1537385831845001m4YX","externalID":"1537385831845001m4YX","code":"THT6700","abbreviation":"THT6700","title":"Value Realization with Oracle Customer Success for SaaS","abstract":"Did you know that 80 percent of customers surveyed by Forbes don’t routinely set and track business goals
 ** Received from  200  set of  3 items, out of a total of  203
{"responseCode":"0","responseMessage":"Success","sections":true,"sectionList":[{"sectionId":"1","sectionTitle":"","total":5,"numItems":5,"from":0,"size":50,"items":[{"sessionID":"1538782999069001riSd","externalID":"1538782999069001riSd","code":"TLD6969","abbreviation":"TLD6969","title":"How Leaders of Tomorrow Are Helping Teams Learn The Future, Today","abstract":"Technology has enabled us to achi
 ** Received for session type  1537894888625001RriS  a set of  5 items, out of a total of  5
{"responseCode":"0","responseMessage":"Su

The Data Lake should now contain 44 JSON files - for 22 session types at each of the two conferences. The next cell checks the file system and lists all files in the *Data Lake*.

In [31]:
import pathlib

# define the path
dataLakeDirectory = pathlib.Path(dataLake)

for currentFile in dataLakeDirectory.iterdir():  
    print(currentFile)

datalake/oow2018-sessions_codeone_PKN.json
datalake/oow2018-sessions_oow_ESS.json
datalake/oow2018-sessions_codeone_PRO.json
datalake/oow2018-sessions_oow_TIP.json
datalake/oow2018-sessions_oow_THT.json
datalake/oow2018-sessions_codeone_DEV.json
datalake/oow2018-sessions_codeone_BOF.json
datalake/oow2018-sessions_codeone_BUS.json
datalake/oow2018-sessions_codeone_MTE.json
datalake/oow2018-sessions_oow_PKN.json
datalake/oow2018-sessions_codeone_PRM.json
datalake/oow2018-sessions_codeone_TLD.json
datalake/oow2018-sessions_oow_PRM.json
datalake/oow2018-sessions_codeone_HOL.json
datalake/oow2018-sessions_codeone_KEY.json
datalake/oow2018-sessions_oow_TRN.json
datalake/oow2018-sessions_oow_HOM.json
datalake/oow2018-sessions_codeone_BQS.json
datalake/oow2018-sessions_oow_TLD.json
datalake/oow2018-sessions_codeone_TRN.json
datalake/oow2018-sessions_oow_KEY.json
datalake/oow2018-sessions_codeone_ESS.json
datalake/oow2018-sessions_oow_CAS.json
datalake/oow2018-sessions_oow_FLP.json
datalake/oow

As a final check, let's load the contents from a randomly selected file into a Pandas Data Frame. If that is successful, we consider the data gathering stage complete. Time for some `data wrangling` in <a href="./2-OOW2018 Session Catalog - Wrangling I - one single, refined, reshaped file.ipynb" target="_new">Step 2 - Data Wrangling</a>.

In [32]:
#as a test, try to load data from one of the generated files 
conference = 'oow' # could also be codeone
sessionType = 'HOL' # could also be one of 21 other values such as TUT, DEV, GEN, BOF,...
sessionPandas = pd.read_json("{0}oow2018-sessions_{1}_{2}.json".format(dataLake, conference, sessionType))
sessionPandas.head(5)

,abbreviation,abstract,allowDoubleBooking,attributevalues,code,codeParts,code_id,es_metadata_id,event,eventCode,...,type_displayorder,type_displayorder_string,useDoubleBooking,useWaitingList,videos,viewAccess,viewAccessPublic,viewFileAccess,waitlistAccess,waitlistLimit
0,HOL6306,"Enhance the security of your Oracle HCM, ERP, ...",0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6306,"{'alpha0': 'HOL', 'numeric1': '6306'}",hol6306,15311715163230013Hbl,Oracle OpenWorld,oow18,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
1,HOL6286,Oracle Data Integration Platform Cloud allows ...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6286,"{'alpha0': 'HOL', 'numeric1': '6286'}",hol6286,1530813370901001gB2W,Oracle OpenWorld,oow18,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
10,HOL6325,In this session learn to develop and deploy a ...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6325,"{'alpha0': 'HOL', 'numeric1': '6325'}",hol6325,15313473829130018nP4,Oracle OpenWorld,oow18,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
11,HOL6328,In this session learn about the latest feature...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6328,"{'alpha0': 'HOL', 'numeric1': '6328'}",hol6328,1531350426847001WHKn,Oracle OpenWorld,oow18,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
12,HOL6331,In this hands-on lab learn the basics of Oracl...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",HOL6331,"{'alpha0': 'HOL', 'numeric1': '6331'}",hol6331,1531436407190001W0ep,Oracle OpenWorld,oow18,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0


# Resources
The public website for the session catalog for Oracle OpenWorld 2018: https://events.rainfocus.com/widget/oracle/oow18/catalogoow18? . 

The session catalog for the co-located CodeOne conference is published at https://events.rainfocus.com/widget/oracle/oow18/catalogcodeone18?

The root endpoint for the REST API for session catalog data: https://events.rainfocus.com/api/search.

Dataquest Data Science Blog — Jupyter Notebook for Beginners: A Tutorial https://www.dataquest.io/blog/jupyter-notebook-tutorial/

The Pandas DataFrame – loading, editing, and viewing data in Python - by Shane Lynn https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/